In [6]:
import numpy as np
import math
import time
from numpy.linalg import inv
from numpy.linalg import norm
def driver():
    x0 = np.array([1, 1])
    x1 = np.array([1, -1])
    x2 = np.array([0, 0])
    Nmax = 100
    tol = 1e-10
    t = time.time()
    for j in range(50):
        [xstar,ier,its] = Newton(x0,tol,Nmax)
        [xstar1,ier1,its1] = Newton(x1,tol,Nmax)
        # [xstar2,ier2,its2] = Newton(x2,tol,Nmax)
    elapsed = time.time()-t
    print('Root for initial guess x=1, y=1:', xstar)
    print('Newton: the error message reads:',ier)
    print('Newton: took this many seconds:',elapsed/50)
    print('Netwon: number of iterations is:',its)
    print("")
    print('Root for initial guess x=1, y=-1:', xstar1)
    print('Newton: the error message reads:',ier1)
    print('Newton: took this many seconds:',elapsed/50)
    print('Netwon: number of iterations is:',its1)
    # print('Newton: the error message reads:',ier2)
    # print('Newton: took this many seconds:',elapsed/50)
    # print('Netwon: number of iterations is:',its2)
    print("")
    t = time.time()
    for j in range(20):
        [xstar,ier,its] = LazyNewton(x0,tol,Nmax)
        [xstar1,ier1,its1] = LazyNewton(x1,tol,Nmax)
    elapsed = time.time()-t
    print('Root for initial guess x=1, y=1:', xstar)
    print('Lazy Newton: the error message reads:',ier)
    print('Lazy Newton: took this many seconds:',elapsed/20)
    print('Lazy Newton: number of iterations is:',its)
    print('')
    print('Root for initial guess x=1, y=-1:', xstar1)
    print('Lazy Newton: the error message reads:',ier1)
    print('Lazy Newton: took this many seconds:',elapsed/20)
    print('Lazy Newton: number of iterations is:',its1)
    print('')
    t = time.time()
    for j in range(20):
        [xstar,ier,its] = Broyden(x0, tol,Nmax)
        [xstar1,ier1,its1] = Broyden(x1, tol,Nmax)
    elapsed = time.time()-t
    print('Root for initial guess x=1, y=1:', xstar)
    print('Broyden: the error message reads:',ier)
    print('Broyden: took this many seconds:',elapsed/20)
    print('Broyden: number of iterations is:',its)
    print('')
    print('Root for initial guess x=1, y=-1:', xstar1)
    print('Broyden: the error message reads:',ier1)
    print('Broyden: took this many seconds:',elapsed/20)
    print('Broyden: number of iterations is:',its1)
    
    
def evalF(x):
    F = np.zeros(2)
    F[0] = x[0]**2 + x[1]**2 - 4
    F[1] = np.exp(x[0]) + x[1] - 1
    return F

def evalJ(x):
    J = np.array([[2*x[0], 2*x[1]], [np.exp(x[0]), 1]])
    return J

def Newton(x0,tol,Nmax):
    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''
    for its in range(Nmax):
        J = evalJ(x0)
        Jinv = inv(J)
        F = evalF(x0)
        x1 = x0 - np.dot(Jinv, F)
        if (norm(x1-x0) < tol):
            xstar = x1
            ier =0
            return[xstar, ier, its]
        x0 = x1
    xstar = x1
    ier = 1
    return[xstar,ier,its]


def LazyNewton(x0,tol,Nmax):
    ''' Lazy Newton = use only the inverse of the Jacobian for initial guess'''
    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''
    J = evalJ(x0)
    Jinv = inv(J)
    for its in range(Nmax):
        F = evalF(x0)
        x1 = x0 - Jinv.dot(F)
        if (norm(x1-x0) < tol):
            xstar = x1
            ier =0
            return[xstar, ier,its]
        x0 = x1
    xstar = x1
    ier = 1
    return[xstar,ier,its]

def Broyden(x0,tol,Nmax):
    '''tol = desired accuracy
    Nmax = max number of iterations'''
    '''Sherman-Morrison
    (A+xy^T)^{-1} = A^{-1}-1/p*(A^{-1}xy^TA^{-1})
    where p = 1+y^TA^{-1}Ax'''
    '''In Newton
    x_k+1 = xk -(G(x_k))^{-1}*F(x_k)'''
    '''In Broyden
    x = [F(xk)-F(xk-1)-\hat{G}_k-1(xk-xk-1)
    y = x_k-x_k-1/||x_k-x_k-1||^2'''
    ''' implemented as in equation (10.16) on page 650 of text'''
    '''initialize with 1 newton step'''
    A0 = evalJ(x0)
    v = evalF(x0)
    A = np.linalg.inv(A0)
    s = -A.dot(v)
    xk = x0+s
    for its in range(Nmax):
        '''(save v from previous step)'''
        w = v
        ''' create new v'''
        v = evalF(xk)
        '''y_k = F(xk)-F(xk-1)'''
        y = v-w;
        '''-A_{k-1}^{-1}y_k'''
        z = -A.dot(y)
        ''' p = s_k^tA_{k-1}^{-1}y_k'''
        p = -np.dot(s,z)
        u = np.dot(s,A)
        ''' A = A_k^{-1} via Morrison formula'''
        tmp = s+z
        tmp2 = np.outer(tmp,u)
        A = A+1./p*tmp2
        ''' -A_k^{-1}F(x_k)'''
        s = -A.dot(v)
        xk = xk+s
        if (norm(s)<tol):
            alpha = xk
            ier = 0
            return[alpha,ier,its]
    alpha = xk
    ier = 1
    return[alpha,ier,its]

driver()

Root for initial guess x=1, y=1: [-1.81626407  0.8373678 ]
Newton: the error message reads: 0
Newton: took this many seconds: 0.0004107761383056641
Netwon: number of iterations is: 7

Root for initial guess x=1, y=-1: [ 1.00416874 -1.72963729]
Newton: the error message reads: 0
Newton: took this many seconds: 0.0004107761383056641
Netwon: number of iterations is: 5

Root for initial guess x=1, y=1: [nan nan]
Lazy Newton: the error message reads: 1
Lazy Newton: took this many seconds: 0.0020070791244506834
Lazy Newton: number of iterations is: 99

Root for initial guess x=1, y=-1: [ 1.00416874 -1.72963729]
Lazy Newton: the error message reads: 0
Lazy Newton: took this many seconds: 0.0020070791244506834
Lazy Newton: number of iterations is: 36

Root for initial guess x=1, y=1: [-1.81626407  0.8373678 ]
Broyden: the error message reads: 0
Broyden: took this many seconds: 0.001795196533203125
Broyden: number of iterations is: 12

Root for initial guess x=1, y=-1: [ 1.00416874 -1.72963729]

C:\Users\Conno\AppData\Local\Temp\ipykernel_7760\236872238.py:65: RuntimeWarning: overflow encountered in exp
  F[1] = np.exp(x[0]) + x[1] - 1


In [22]:
def driver1():
    Nmax = 100
    x0= np.array([0,2,2])
    tol = 1e-6
    [xstar,ier,its] = Newton(x0,tol,Nmax)
    print("Newton method found the solution ",xstar)
    print("Newton number of iterations ", its)
    print("ier is ", ier )
    print("")
    [xstar,gval,ier, its] = SteepestDescent(x0,tol,Nmax)
    print("the steepest descent code found the solution ",xstar)
    print("g evaluated at this point is ", gval)
    print("Steepest Decent number of iterations ", its)
    print("ier is ", ier )
    
    
###########################################################
#functions:
def evalF(x):
    F = np.zeros(3)
    F[0] = x[0] +math.cos(x[0]*x[1]*x[2])-1.
    F[1] = (1.-x[0])**(0.25) + x[1] +0.05*x[2]**2 -0.15*x[2]-1
    F[2] = -x[0]**2-0.1*x[1]**2 +0.01*x[1]+x[2] -1
    return F
def evalJ(x):
    J = np.array([[1.+x[1]*x[2]*math.sin(x[0]*x[1]*x[2]),x[0]*x[2]*math.sin(x[0]*x[1]*x[2]
    ),x[1]*x[0]*math.sin(x[0]*x[1]*x[2])],
    [-0.25*(1-x[0])**(-0.75),1,0.1*x[2]-0.15],
    [-2*x[0],-0.2*x[1]+0.01,1]])
    return J
    
def evalg(x):
    F = evalF(x)
    g = F[0]**2 + F[1]**2 + F[2]**2
    return g
    
def eval_gradg(x):
    F = evalF(x)
    J = evalJ(x)
    gradg = np.transpose(J).dot(F)
    return gradg

###############################
### newton method code
def Newton(x0,tol,Nmax):
    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''
    for its in range(Nmax):
        J = evalJ(x0)
        Jinv = inv(J)
        F = evalF(x0)
        x1 = x0 - np.dot(Jinv, F)
        if (norm(x1-x0) < tol):
            xstar = x1
            ier =0
            return[xstar, ier, its]
        x0 = x1
    xstar = x1
    ier = 1
    return[xstar,ier,its]
    
    
###############################
### steepest descent code
def SteepestDescent(x,tol,Nmax):
    for its in range(Nmax):
        g1 = evalg(x)
        z = eval_gradg(x)
        z0 = norm(z)
        if z0 == 0:
            print("zero gradient")
        z = z/z0
        alpha1 = 0
        alpha3 = 1
        dif_vec = x - alpha3*z
        g3 = evalg(dif_vec)
        
        while g3>=g1:
            alpha3 = alpha3/2
            dif_vec = x - alpha3*z
            g3 = evalg(dif_vec)
            
        if alpha3<tol:
            print("no likely improvement")
            ier = 0
            return [x,g1,ier,its]
        
        alpha2 = alpha3/2
        dif_vec = x - alpha2*z
        g2 = evalg(dif_vec)
        
        h1 = (g2 - g1)/alpha2
        h2 = (g3-g2)/(alpha3-alpha2)
        h3 = (h2-h1)/alpha3
        
        alpha0 = 0.5*(alpha2 - h1/h3)
        dif_vec = x - alpha0*z
        g0 = evalg(dif_vec)
        
        if g0<=g3:
            alpha = alpha0
            gval = g0
        else:
            alpha = alpha3
            gval =g3
            
        x = x - alpha*z
        
        if abs(gval - g1)<tol:
            ier = 0
            return [x,gval,ier, its]
        
        
    print('max iterations exceeded')
    ier = 1
    return [x,g1,ier,its]

driver1()

Newton method found the solution  [0.  0.1 1. ]
Newton number of iterations  4
ier is  0

the steepest descent code found the solution  [6.68766613e-05 1.00031501e-01 1.00001477e+00]
g evaluated at this point is  4.878569704219945e-09
Steepest Decent number of iterations  6
ier is  0


In [23]:
def driver2():
    Nmax = 100
    x0= np.array([0,2,2])
    tol1 = 5e-2
    [xstar_steep,gval,ier, its] = SteepestDescent(x0,tol1,Nmax)
    
    tol2 = 1e-6
    [xstar,ier,its1] = Newton(xstar_steep,tol2,Nmax)
    print("Hybrid method found the solution ",xstar)
    print("Hybrid number of iterations ", its+its1)
    print("ier is ", ier )
    
driver2()


Hybrid method found the solution  [1.02738975e-16 1.00000000e-01 1.00000000e+00]
Hybrid number of iterations  5
ier is  0
